# Preprocessing Data

This automated script is designed to take surface water outputs from HEC-RAS. This section shows the necessary preprocessing steps for the surface water data before it can be utilized in the modeling packages of ModFlow6 with FloPy.

In [1]:
#-----------------------Importing Libraries-----------------------#
# import os
# import flopy
# import pathlib
# import matplotlib.pyplot as plt
# import numpy as np
# import geopandas as gpd
# import pandas as pd
# import rasterio
# import pyproj
# import shutil
# import random
# import jupyter_book
# import pathlib as pl
# from pathlib import Path
# from rasterio.crs import CRS
# from rasterio.plot import show
# from rasterio.warp import calculate_default_transform, reproject, Resampling
# from rasterio.transform import from_bounds
# from rasterio.transform import rowcol
# from rasterio.mask import mask
# from shapely.geometry import box, Point, Polygon, LineString
# from flopy.utils.binaryfile import HeadFile
# from scipy.interpolate import griddata
# from pprint import pformat
# from flopy.plot.styles import styles
# from matplotlib.lines import Line2D
# from flopy.mf6 import MFSimulation
# from matplotlib import cbook, cm
# from matplotlib.colors import LightSource
# from modflow_devtools.misc import get_env, timed

%run /Users/u4eeevmq/Documents/Python/HyporheicFloPy/VQuintana/common_imports.py


In [2]:
#---------------------- Directories ----------------------#
## Data Paths
water_surface_elevation_raster = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\CH00365\GMS\WSE (Max).USGS_1m_FebApril2018_DEM.U_USGS-3DEP_dtm_hyrdo_flattened_20240412.2041_1.tif"
terrain_elevation_raster = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\CH00365\GMS\BlendedTerrain.LoweredTerrain.tif"
ground_water_domain_shapefile = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\InputShapefiles\GWDomain.shp"
left_boundary_floodplain = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\InputShapefiles\L_FPL.shp"
right_boundary_floodplain = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\InputShapefiles\R_FPL.shp"

## Projection File Path
projection_file = r"C:\Users\u4eeevmq\Documents\Python\test_12_18_2024\CH00365\RAS\GIS_Data\102739_TX_central.prj"

### NOTE: This script can be adapted to run groundwater modeling simulations for other locations by changing the directories above to designated inputs

In [3]:
#--------------------------------- Load Projection FIle -----------------------------------------#
## HEC-RAS Projection File
hec_ras_crs = CRS.from_string(open(projection_file).read().strip())

## Store the CRS
%store hec_ras_crs
### NOTE: If the projection file is not available, the CRS can be defined manually here.

Stored 'hec_ras_crs' (CRS)


In [4]:
#---------------------------------Terrain Elevation -----------------------------------------#
# Open the terrain raster
with rasterio.open(terrain_elevation_raster) as src:
    terrain_elevation = src.read(1)
    raster_transform = src.transform
    raster_crs = src.crs

    # Calculate the new transform and dimensions
    transform, width, height = calculate_default_transform(
        raster_crs, hec_ras_crs, src.width, src.height, *src.bounds
    )

    # Define metadata for the new raster
    new_meta = src.meta.copy()
    new_meta.update({
        "crs": hec_ras_crs,
        "transform": transform,
        "width": width,
        "height": height
    })

    # Define output file name
    output_raster = "reprojected_terrain_raster.tif"

    # Reproject and save
    with rasterio.open(output_raster, "w", **new_meta) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=raster_transform,
            src_crs=raster_crs,
            dst_transform=transform,
            dst_crs=hec_ras_crs,
            resampling=Resampling.nearest
        )

print(f"Reprojected raster saved as {output_raster}")

## Store the terrain elevation raster
%store terrain_elevation
%store raster_transform
%store transform
%store raster_crs
%store output_raster

Reprojected raster saved as reprojected_terrain_raster.tif
Stored 'terrain_elevation' (ndarray)
Stored 'raster_transform' (Affine)
Stored 'transform' (Affine)
Stored 'raster_crs' (CRS)
Stored 'output_raster' (str)


In [5]:
#------------------------------Model Grid--------------------------------#
# Open the terrain raster
with rasterio.open(terrain_elevation_raster) as src:
    terrain_elevation = src.read(1)
    raster_transform = src.transform
    raster_crs = src.crs

    # Calculate the new transform and dimensions
    transform, width, height = calculate_default_transform(
        raster_crs, hec_ras_crs, src.width, src.height, *src.bounds
    )

    # Define metadata for the new raster
    new_meta = src.meta.copy()
    new_meta.update({
        "crs": hec_ras_crs,
        "transform": transform,
        "width": width,
        "height": height
    })

    # Define output file name
    output_raster = "reprojected_terrain_raster.tif"

    # Reproject and save
    with rasterio.open(output_raster, "w", **new_meta) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=raster_transform,
            src_crs=raster_crs,
            dst_transform=transform,
            dst_crs=hec_ras_crs,
            resampling=Resampling.nearest
        )

print(f"Reprojected raster saved as {output_raster}")

## Store the terrain elevation raster
%store terrain_elevation
%store raster_transform
%store transform
%store raster_crs
%store output_raster

Reprojected raster saved as reprojected_terrain_raster.tif
Stored 'terrain_elevation' (ndarray)
Stored 'raster_transform' (Affine)
Stored 'transform' (Affine)
Stored 'raster_crs' (CRS)
Stored 'output_raster' (str)


In [6]:
#--------------------------------- Water Surface Elevation -----------------------------------------#
# Open the input raster
with rasterio.open(water_surface_elevation_raster) as src:
    surface_elevation = src.read(1)
    raster_transform = src.transform
    raster_crs = src.crs

    # Calculate the new transform and dimensions
    transform, width, height = calculate_default_transform(
        raster_crs, hec_ras_crs, src.width, src.height, *src.bounds
    )
    
    # Define metadata for the new raster
    new_meta = src.meta.copy()
    new_meta.update({
        "crs": hec_ras_crs,
        "transform": transform,
        "width": width,
        "height": height
    })

    # Define output file name
    output_raster = "reprojected_water_surface_elevation_raster.tif"

    # Reproject and save
    with rasterio.open(output_raster, "w", **new_meta) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=raster_transform,
            src_crs=raster_crs,
            dst_transform=transform,
            dst_crs=hec_ras_crs,
            resampling=Resampling.nearest
        )

print(f"Reprojected raster saved as {output_raster}")

# Crop the water surface elevation raster to the terrain extent
with rasterio.open(output_raster) as src:
    terrain_bounds = src.bounds
    terrain_geom = box(*terrain_bounds)

    with rasterio.open(water_surface_elevation_raster) as src:
        out_image, out_transform = mask(src, [terrain_geom], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        cropped_output_raster = "cropped_water_surface_elevation_raster.tif"
        with rasterio.open(cropped_output_raster, "w", **out_meta) as dst:
            dst.write(out_image)

print(f"Cropped water surface elevation raster saved as {cropped_output_raster}")

%store cropped_output_raster

Reprojected raster saved as reprojected_water_surface_elevation_raster.tif
Cropped water surface elevation raster saved as cropped_water_surface_elevation_raster.tif
Stored 'cropped_output_raster' (str)


In [7]:
#--------------------------------- Shapefiles -----------------------------------------#
## Load and reproject shapefiles to match the raster CRS
ground_water_domain = gpd.read_file(ground_water_domain_shapefile).to_crs(hec_ras_crs)
left_boundary = gpd.read_file(left_boundary_floodplain).to_crs(hec_ras_crs)
right_boundary = gpd.read_file(right_boundary_floodplain).to_crs(hec_ras_crs)

## Save the reprojected shapefiles
%store ground_water_domain
%store left_boundary
%store right_boundary

Stored 'ground_water_domain' (GeoDataFrame)
Stored 'left_boundary' (GeoDataFrame)
Stored 'right_boundary' (GeoDataFrame)
